In [7]:
pip install rasterio numpy pandas tqdm pvlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import numpy as np
import rasterio
from rasterio.vrt import WarpedVRT
import os

def calculate_slope_aspect(dem, transform):
    # ... (생략: 기존 구현 그대로)
    x, y = np.gradient(dem, 30, 30)
    slope = np.pi/2 - np.arctan(np.hypot(x, y))
    aspect = np.arctan2(-x, y)
    aspect = np.where(aspect < 0, 2*np.pi + aspect, aspect)
    return slope, aspect

def generate_hillshade_tifs(input_dem, slope_tif, aspect_tif, target_crs="EPSG:32652"):
    # 1) DEM → 가상 UTM(메모리 reprojection)
    with rasterio.open(input_dem) as src:
        vrt_params = {
            "crs": target_crs,
            "resampling": rasterio.enums.Resampling.bilinear,
            "resolution": (30, 30)
        }
        with WarpedVRT(src, **vrt_params) as vrt:
            dem = vrt.read(1).astype("float64")
            transform = vrt.transform

            # 2) slope, aspect 계산
            slope_rad, aspect_rad = calculate_slope_aspect(dem, transform)

            # 라디안 → 도 단위 변환
            slope_deg  = np.degrees(slope_rad)         # 0°~90°
            aspect_deg = np.degrees(aspect_rad) % 360  # 0°~360°

            # 이제 slope_deg, aspect_deg 를 GeoTIFF 로 저장하거나
            # 힐셰이드·분석 등에 바로 활용하세요.
            # 3) 메타데이터 준비 (driver만 GTiff 로 강제)
            meta = vrt.meta.copy()
            meta.update({
                "driver": "GTiff",     # ← 이 줄이 필수
                "dtype": "float32",
                "count": 1,
                "nodata": None
            })

            # 4) slope 저장
            with rasterio.open(slope_tif, "w", **meta) as dst:
                dst.write(slope_deg.astype("float32"), 1)

            # 5) aspect 저장
            with rasterio.open(aspect_tif, "w", **meta) as dst:
                dst.write(aspect_deg.astype("float32"), 1)

    print("완료: Slope & Aspect GeoTIFF 생성")

# 사용 예시
input_dem  = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Gangneung_Clip.tif"
slope_tif  = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Gangneung_Slope.tif"
aspect_tif = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Gangneung_Aspect.tif"
generate_hillshade_tifs(input_dem, slope_tif, aspect_tif)

완료: Slope & Aspect GeoTIFF 생성


In [5]:
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import xy
from tqdm import tqdm
from datetime import datetime
from pvlib import solarposition

def calculate_slope_aspect(dem, transform):
    # 경사도(slope), 방향(aspect) 계산 (단위: 라디안)
    x, y = np.gradient(dem, 30, 30)  # 30m 해상도 기준
    slope = np.pi/2 - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    aspect = np.where(aspect < 0, 2 * np.pi + aspect, aspect)
    return slope, aspect

def compute_hillshade(slope, aspect, azimuth_deg, altitude_deg):
    # 태양 방위각, 고도각 → 라디안 변환
    azimuth_rad = np.radians(azimuth_deg)
    altitude_rad = np.radians(altitude_deg)
    
    shaded = (np.sin(altitude_rad) * np.sin(slope) +
              np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect))
    
    shaded = np.clip(shaded, 0, 1)  # 0~1 범위로 정리
    return shaded * 255  # 🎯 0~255 범위로 변환

def generate_hourly_hillshades(dem_path, solar_csv_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # DEM 로드
    with rasterio.open(dem_path) as dem_src:
        dem = dem_src.read(1)
        meta = dem_src.meta.copy()
        transform = dem_src.transform
    
    # slope & aspect 계산
    slope, aspect = calculate_slope_aspect(dem, transform)

    # 태양 위치 데이터 로드
    solar_df = pd.read_csv(solar_csv_path, parse_dates=['datetime'])

    # Hillshade 계산
    for _, row in tqdm(solar_df.iterrows(), total=len(solar_df)):
        dt = row['datetime']
        elev = row['elevation']
        azim = row['azimuth']

        if elev <= 0:
            continue  # 태양이 떠 있지 않으면 저장하지 않음

        hillshade = compute_hillshade(slope, aspect, azim, elev)  # 0~255

        # 저장 경로 생성
        fname = f"hillshade_{dt.strftime('%Y-%m-%d_%H-%M')}.tif"
        out_path = os.path.join(output_dir, fname)

        # 저장 (GeoTIFF, uint8, WGS84)
        meta.update(dtype='uint8', count=1)
        with rasterio.open(out_path, 'w', **meta) as dst:
            dst.write(hillshade.astype('uint8'), 1)

    print(f"Hillshade 생성 완료 (0~255 스케일, 태양이 뜬 시간만): {output_dir}")



# ✅ 사용 예시 (경로 설정)
dem_path = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Naju_DEM_Clip.tif"
solar_csv_path = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\solar_position_naju.csv"
output_dir = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Hillshade_naju"

generate_hourly_hillshades(dem_path, solar_csv_path, output_dir)


100%|██████████| 8760/8760 [00:31<00:00, 278.61it/s]

Hillshade 생성 완료 (0~255 스케일, 태양이 뜬 시간만): C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Hillshade_naju


In [ ]:
#Hillshade_Band approach
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import xy
from affine import Affine
from pvlib.location import Location
from datetime import datetime
from tqdm import tqdm


def calculate_slope_aspect(dem):
    x, y = np.gradient(dem, 90, 90)
    slope = np.pi / 2 - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    aspect = np.where(aspect < 0, 2 * np.pi + aspect, aspect)
    return slope, aspect


def compute_hillshade(slope, aspect, azimuth_deg, altitude_deg):
    azimuth_rad = np.radians(azimuth_deg)
    altitude_rad = np.radians(altitude_deg)
    shaded = (np.sin(altitude_rad) * np.sin(slope) +
              np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect))
    shaded = np.clip(shaded, 0, 1)
    return (shaded * 255).astype(np.uint8)


def get_lat_band_masks(dem_data, transform, bands):
    rows, cols = dem_data.shape
    lats = np.empty((rows, cols))
    for r in range(rows):
        _, lat_row = xy(transform, r, 0)
        lats[r, :] = lat_row  # 모든 열에 동일한 위도
    return {b: (lats >= b[0]) & (lats < b[1]) for b in bands}


def get_solar_position(lat, lon, time):
    loc = Location(latitude=lat, longitude=lon)
    solpos = loc.get_solarposition(time)
    return 90 - solpos['apparent_zenith'].values[0], solpos['azimuth'].values[0]


def generate_combined_hillshades(dem_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        transform = src.transform
        meta = src.meta.copy()

    meta.update(dtype='uint8', count=1, nodata=0)

    slope, aspect = calculate_slope_aspect(dem)
    bands = [(33, 34), (34, 35), (35, 36), (36, 37), (37, 38)]
    masks = get_lat_band_masks(dem, transform, bands)

    times = pd.date_range(start='2023-01-01 00:00', end='2023-12-31 23:00', freq='1H', tz='Asia/Seoul')

    for dt in tqdm(times, desc="Generating hillshades"):
        combined = np.zeros_like(dem, dtype='uint8')

        for (lat_min, lat_max), mask in masks.items():
            if not np.any(mask):
                continue

            center_lat = (lat_min + lat_max) / 2
            center_lon = 126.5
            elev, azim = get_solar_position(center_lat, center_lon, dt)

            if elev <= 0:
                continue

            hs = compute_hillshade(slope, aspect, azim, elev)
            combined = np.where(mask, hs, combined)

        fname = f"hillshade_combined_{dt.strftime('%Y-%m-%d_%H-%M')}.tif"
        out_path = os.path.join(output_dir, fname)

        with rasterio.open(out_path, 'w', **meta) as dst:
            dst.write(combined, 1)

    print(f"\n🎉 모든 위도 Band에 대해 Hillshade 생성 완료: {output_dir}")

dem_path = r"C:\Users\user\Desktop\Junkyo\2025\WREC\Korean Peninsula\Korea90m_GRS80.img"
output_dir = r"C:\Users\user\Desktop\Junkyo\2025\WREC\Korea_hillshade"

generate_combined_hillshades(dem_path, output_dir)

✅ 저장 완료: hillshade_combined_2020-06-21_05-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_06-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_07-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_08-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_09-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_10-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_11-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_12-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_13-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_14-00.tif
✅ 저장 완료: hillshade_combined_2020-06-21_15-00.tif


KeyboardInterrupt: 

In [ ]:
#Hillshade 병렬
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import xy
from affine import Affine
from pvlib.location import Location
from datetime import datetime
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed

def calculate_slope_aspect(dem):
    x, y = np.gradient(dem, 90, 90)
    slope = np.pi / 2 - np.arctan(np.sqrt(x * x + y * y))
    aspect = np.arctan2(-x, y)
    aspect = np.where(aspect < 0, 2 * np.pi + aspect, aspect)
    return slope, aspect

def compute_hillshade(slope, aspect, azimuth_deg, altitude_deg):
    azimuth_rad = np.radians(azimuth_deg)
    altitude_rad = np.radians(altitude_deg)
    shaded = (np.sin(altitude_rad) * np.sin(slope) +
              np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect))
    shaded = np.clip(shaded, 0, 1)
    return (shaded * 255).astype(np.uint8)

def get_latitude_band_mask(dem_data, transform, lat_min, lat_max):
    rows, cols = dem_data.shape
    mask = np.zeros_like(dem_data, dtype=bool)
    for r in range(rows):
        for c in range(cols):
            lon, lat = xy(transform, r, c)
            if lat_min <= lat < lat_max:
                mask[r, c] = True
    return mask

def get_solar_position(lat, lon, time):
    loc = Location(latitude=lat, longitude=lon)
    if not isinstance(time, pd.DatetimeIndex):
        time = pd.DatetimeIndex([time])  # 수정된 부분
    solpos = loc.get_solarposition(time)
    zenith = solpos.iloc[0]['apparent_zenith']
    azimuth = solpos.iloc[0]['azimuth']
    return 90 - zenith, azimuth  # 고도각, 방위각


def process_hour(args):
    dt, dem, slope, aspect, transform, meta, band_ranges, out_dir = args
    combined_hillshade = np.zeros_like(dem, dtype='uint8')

    for lat_min, lat_max in band_ranges:
        mask = get_latitude_band_mask(dem, transform, lat_min, lat_max)
        if not np.any(mask):
            continue

        center_lat = (lat_min + lat_max) / 2
        center_lon = 126.5
        elev, azim = get_solar_position(center_lat, center_lon, [dt])

        if elev <= 0:
            continue  # 태양이 지평선 아래

        slope_masked = np.where(mask, slope, 0)
        aspect_masked = np.where(mask, aspect, 0)
        hillshade = compute_hillshade(slope_masked, aspect_masked, azim, elev)
        hillshade = np.where(mask, hillshade, 0)

        combined_hillshade = np.where(mask, hillshade, combined_hillshade)

    # 저장
    fname = f"hillshade_combined_{dt.strftime('%Y-%m-%d_%H-%M')}.tif"
    out_path = os.path.join(out_dir, fname)
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(combined_hillshade, 1)

    return fname

def generate_combined_hillshades_parallel(dem_path, output_dir, num_workers=8):
    os.makedirs(output_dir, exist_ok=True)

    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        transform = src.transform
        meta = src.meta.copy()

    meta.update(dtype='uint8', count=1, nodata=0)

    slope, aspect = calculate_slope_aspect(dem)

    band_ranges = [(33, 34), (34, 35), (35, 36), (36, 37), (37, 38)]
    times = pd.date_range(start='2023-01-01 00:00', end='2023-12-31 23:00', freq='1h', tz='Asia/Seoul')

    tasks = [(dt, dem, slope, aspect, transform, meta, band_ranges, output_dir) for dt in times]

    print(f"⏱ 병렬처리 시작: {len(tasks)}시간 × {len(band_ranges)}밴드")

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(process_hour, task) for task in tasks]
        for f in tqdm(as_completed(futures), total=len(futures)):
            _ = f.result()

    print(f"\n✅ 병렬 Hillshade 계산 완료! 출력 폴더: {output_dir}")

dem_path = r"C:\Users\user\Desktop\Junkyo\2025\WREC\Korean Peninsula\Korea90m_GRS80.img"
output_dir = r"C:\Users\user\Desktop\Junkyo\2025\WREC\Korea_hillshade"

generate_combined_hillshades(dem_path, output_dir)

✅ 저장 완료: hillshade_combined_2020-06-21_05-00.tif


KeyboardInterrupt: 

In [5]:
import os
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import geopandas as gpd

# 1) 파일 경로
shp_path      = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\naju_road_Hori.shp"
raster_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Hillshade_naju"
output_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\mask_naju_Hori"
os.makedirs(output_folder, exist_ok=True)

# 2) shapefile 불러오기 + CRS 지정
gdf = gpd.read_file(shp_path)

# 이미 CRS가 올바르게 기록돼 있지 않다면 강제 지정
# “Korean 1985 / Modified Central Belt” = EPSG:2097
if gdf.crs is None or gdf.crs.to_epsg() != 2097:
    gdf = gdf.set_crs(epsg=2097, allow_override=True)

for fname in os.listdir(raster_folder):
    if not fname.lower().endswith(".tif"):
        continue

    in_rast  = os.path.join(raster_folder, fname)
    out_rast = os.path.join(output_folder, f"masked_{fname}")

    with rasterio.open(in_rast) as src:
        # 3) shapefile → raster CRS 로 변환
        vect = gdf.to_crs(src.crs)

        # GeoJSON 포맷으로 변환
        geoms = [mapping(geom) for geom in vect.geometry]

        # 4) 마스킹 (픽셀의 중심이 폴리곤 내에 있어야 내부로 간주됨)
        out_img, out_trans = mask(
            src,
            geoms,
            all_touched=False,  
            crop=False,
            nodata=0,
            filled=True
        )

        # 5) 메타데이터 갱신 및 저장
        meta = src.meta.copy()
        meta.update({
            "driver":  "GTiff",
            "height":  out_img.shape[1],
            "width":   out_img.shape[2],
            "transform": out_trans,
            "nodata":  0
        })

        with rasterio.open(out_rast, "w", **meta) as dst:
            dst.write(out_img)

print("✅ 마스킹 완료!")

✅ 마스킹 완료!


In [6]:
import os
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# 1) 파일 경로 설정
shp_path      = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\gangneung_road_Hori.shp"
raster_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Hillshade_gangneung"
output_csv    = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\gangneung_Hori_zerocell.csv"

# 2) shapefile 불러오기 및 CRS 지정 (Korean 1985 / Modified Central Belt, EPSG:2097)
gdf = gpd.read_file(shp_path)
if gdf.crs is None or gdf.crs.to_epsg() != 2097:
    gdf = gdf.set_crs(epsg=2097, allow_override=True)

# 3) 각 래스터 파일에 대해 마스킹 처리 후 통계 계산 (폴리곤 외부는 null값(np.nan)으로 처리)
stats_list = []  # 결과 통계를 저장할 리스트
raster_files = sorted([f for f in os.listdir(raster_folder) if f.lower().endswith('.tif')])

for fname in tqdm(raster_files, desc="래스터 처리중"):
    raster_path = os.path.join(raster_folder, fname)
    
    with rasterio.open(raster_path) as src:
        # shapefile을 해당 래스터의 CRS로 재투영 후, GeoJSON 포맷으로 변환
        vect = gdf.to_crs(src.crs)
        geoms = [mapping(geom) for geom in vect.geometry]
        
        # 마스킹: 픽셀의 중심이 폴리곤 내에 있어야 내부로 간주
        # filled=False로 해서 MaskedArray 형태로 받습니다.
        out_img, out_trans = mask(
            src,
            geoms,
            all_touched=False,
            crop=False,
            filled=False
        )
        
    # out_img는 (1, rows, cols) 형태의 MaskedArray임.
    masked = out_img[0]
    
    # 먼저 float32 타입으로 변환하고, 마스크된 영역을 np.nan으로 할당
    data = masked.astype("float32")
    data[masked.mask] = np.nan
    
    # 유효한 셀(즉, np.nan이 아닌 셀) 수 계산
    valid_count = np.count_nonzero(~np.isnan(data))
    # 값이 0인 셀 수 계산 (np.nan은 비교 대상에서 제외)
    zero_count = np.count_nonzero(data == 0)
    
    stats_list.append([fname, valid_count, zero_count])

# 4) 결과 통계를 CSV 파일로 저장 (A열: 파일명, B열: 유효 셀 수, C열: 0인 셀 수)
stats_df = pd.DataFrame(stats_list, columns=['RasterFile', 'ValidCellCount', 'ZeroCellCount'])
stats_df.to_csv(output_csv, index=False, encoding='utf-8')

print("CSV 파일이 생성되었습니다:", output_csv)

래스터 처리중:   0%|          | 0/4400 [00:00<?, ?it/s]

래스터 처리중: 100%|██████████| 4400/4400 [00:42<00:00, 104.09it/s]

CSV 파일이 생성되었습니다: C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\gangneung_Hori_zerocell.csv


In [28]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping

# 1) 입력 파일 경로
shp_path    = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\gangneung_road_Vert.shp"
raster_path = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Gangneung_Slope.tif"
output_path = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Gangneung_Slope_Vert.tif"

# 2) shapefile 불러오기 + CRS 강제 지정(EPSG:2097)
gdf = gpd.read_file(shp_path)
if gdf.crs is None or gdf.crs.to_epsg() != 2097:
    gdf = gdf.set_crs(epsg=2097, allow_override=True)

# 3) rasterio로 래스터 열기 → 벡터를 래스터 CRS에 맞춰 재투영 → GeoJSON 리스트
with rasterio.open(raster_path) as src:
    vect = gdf.to_crs(src.crs)
    geoms = [mapping(geom) for geom in vect.geometry]

    # 4) 마스킹: all_touched=True, 내부 보존·외부 0
    out_img, out_trans = mask(
        src,
        geoms,
        all_touched=True,
        crop=False,
        nodata=0,
        filled=True
    )

    # 5) 메타데이터 갱신 (driver는 GTiff로)
    meta = src.meta.copy()
    meta.update({
        "driver":   "GTiff",
        "height":   out_img.shape[1],
        "width":    out_img.shape[2],
        "transform":out_trans,
        "nodata":   0
    })

# 6) 결과 쓰기
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with rasterio.open(output_path, "w", **meta) as dst:
    dst.write(out_img)

print("✅ 단일 래스터 마스킹 완료:", output_path)

✅ 단일 래스터 마스킹 완료: C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Gangneung_Slope_Vert.tif


In [13]:
import os
import numpy as np
import rasterio
import pandas as pd
from tqdm import tqdm

def process_raster_file(raster_path, SI, shading_factor, output_path):
    """
    한 개의 래스터 파일에 대해 처리:
      - 셀 값이 0이면 0, 그렇지 않으면 계산된 발전량으로 채움
      - 발전량 = 0.22 * SI * (1 - shading_factor) * 450
      - 반환값: non_zero_count, total_value
    """
    with rasterio.open(raster_path) as src:
        arr = src.read(1)  # 단일 밴드 읽기
        meta = src.meta.copy()
    
    # 계산: 모든 non-zero 셀에 대해 동일한 발전량을 계산합니다.
    gen_value = 0.22 * SI * (1 - shading_factor) * 450
    result = np.where(arr == 0, 0, gen_value)

    # 통계 값 계산: 셀이 0이 아닌 셀들의 갯수와, 전체 셀의 값 합계
    nonzero_count = np.count_nonzero(result)
    total_value = np.sum(result)

    # 메타데이터 업데이트 (GeoTIFF 쓰기용)
    meta.update({
        "dtype": "float32",
        "driver": "GTiff"
    })
    
    # 결과 저장
    with rasterio.open(output_path, "w", **meta) as dst:
        dst.write(result.astype(np.float32), 1)
    
    return nonzero_count, total_value

# ------------------------------------
# 1. 파일 경로 설정
hillshade_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\mask_gangneung_Hori"
csv_path = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\tmy\Study_tmy\Gangneung_samcsv.csv" # CSV 파일 (UTF-8 인코딩)
output_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Power_gangneung_Hori"
os.makedirs(output_folder, exist_ok=True)

# ------------------------------------
# 2. CSV 파일 로드 (헤더가 3번째 행에 있다고 가정)
solar_df = pd.read_csv(csv_path, encoding='utf-8', header=2)

required_columns = ['Irradiance', 'SelfShading']
for col in required_columns:
    if col not in solar_df.columns:
        raise ValueError(f"CSV 파일에 '{col}' 열이 없습니다.")

# ------------------------------------
# 3. 폴더 내의 래스터 파일들을 정렬된 순서대로 읽기
raster_files = sorted([f for f in os.listdir(hillshade_folder) if f.lower().endswith('.tif')])
if len(raster_files) != len(solar_df):
    print("경고: 래스터 파일의 갯수와 CSV 파일의 행 수가 일치하지 않습니다. 둘 중 작은 개수로 진행합니다.")

n_files = min(len(raster_files), len(solar_df))

# ------------------------------------
# 결과 통계 저장 리스트 (각 행: [래스터파일명, 0이 아닌 셀 갯수, 전체 셀값 합계])
stats_list = []

# ------------------------------------
# 4. 각 래스터 파일에 대해 처리
for i in tqdm(range(n_files), desc="래스터 처리중"):
    raster_file = raster_files[i]
    raster_path = os.path.join(hillshade_folder, raster_file)
    
    # CSV의 i번째 행값 추출
    row = solar_df.iloc[i]
    
    # Solar Irradiance (SI) 계산: GHI, DNI, DHI 합산
    SI = row['Irradiance']
    shading_factor = row['SelfShading']
    
    # 출력 파일 경로 생성 (파일명 앞에 "processed_" 접두사 추가)
    output_path = os.path.join(output_folder, f"processed_{raster_file}")
    
    # 한 래스터 파일 처리 및 통계 계산
    nonzero_count, total_value = process_raster_file(raster_path, SI, shading_factor, output_path)
    
    # 통계 정보 리스트에 추가: 파일명, 0이 아닌 셀 갯수, 전체 값의 합계
    stats_list.append([raster_file, nonzero_count, total_value])

# ------------------------------------
# 5. 통계 결과를 새로운 CSV 파일로 저장
stats_df = pd.DataFrame(stats_list, columns=['RasterFile', 'NonZeroCount', 'TotalValue'])
stats_csv_path = os.path.join(output_folder, "raster_statistics.csv")
stats_df.to_csv(stats_csv_path, index=False, encoding='utf-8')

print(f"모든 래스터 파일의 발전량 계산 및 통계 CSV 생성 및 {output_folder} 저장 완료!")

경고: 래스터 파일의 갯수와 CSV 파일의 행 수가 일치하지 않습니다. 둘 중 작은 개수로 진행합니다.


래스터 처리중: 100%|██████████| 4400/4400 [01:46<00:00, 41.13it/s]

모든 래스터 파일의 발전량 계산 및 통계 CSV 생성 및 C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Power_gangneung_Hori 저장 완료!


In [ ]:
import os
import numpy as np
import rasterio
import pandas as pd
from tqdm import tqdm

def process_raster_file(raster_path, SI, shading_factor):
    """
    한 개의 래스터 파일에 대해 처리:
      - 셀 값이 0이면 0, 그렇지 않으면 계산된 발전량으로 채움
      - 발전량 = 0.22 * SI * (1 - shading_factor) * 450
      - 반환값: non_zero_count, total_value
    """
    with rasterio.open(raster_path) as src:
        arr = src.read(1)  # 단일 밴드 읽기
    
    # 발전량 계산: 모든 non-zero 셀에 대해 동일한 값
    gen_value = 0.9* 0.22 * SI * (shading_factor) * 450
    result = np.where(arr == 0, 0, gen_value)
    
    # 통계 값 계산: 셀이 0이 아닌 셀들의 갯수와, 전체 셀의 합계
    nonzero_count = np.count_nonzero(result)
    total_value = np.sum(result)
    
    return nonzero_count, total_value

# ------------------------------------
# 1. 파일 경로 설정
hillshade_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\mask_naju_Hori"
csv_path = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\tmy\Study_tmy\Naju_samcsv.csv"  # CSV 파일 (UTF-8 인코딩)
output_folder = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Power_naju_Hori"
os.makedirs(output_folder, exist_ok=True)

# ------------------------------------
# 2. CSV 파일 로드 (헤더가 3번째 행에 있다고 가정)
solar_df = pd.read_csv(csv_path, encoding='utf-8', header=2)

required_columns = ['Irradiance', 'Vable']
for col in required_columns:
    if col not in solar_df.columns:
        raise ValueError(f"CSV 파일에 '{col}' 열이 없습니다.")

# ------------------------------------
# 3. 폴더 내의 래스터 파일들을 정렬된 순서대로 읽기
raster_files = sorted([f for f in os.listdir(hillshade_folder) if f.lower().endswith('.tif')])
if len(raster_files) != len(solar_df):
    print("경고: 래스터 파일의 갯수와 CSV 파일의 행 수가 일치하지 않습니다. 둘 중 작은 개수로 진행합니다.")

n_files = min(len(raster_files), len(solar_df))

# ------------------------------------
# 결과 통계 저장 리스트 (각 행: [래스터파일명, 0이 아닌 셀 갯수, 전체 셀값 합계])
stats_list = []

# ------------------------------------
# 4. 각 래스터 파일에 대해 처리 (tif 파일은 생성하지 않고 통계만 계산)
for i in tqdm(range(n_files), desc="래스터 처리중"):
    raster_file = raster_files[i]
    raster_path = os.path.join(hillshade_folder, raster_file)
    
    # CSV의 i번째 행값 추출
    row = solar_df.iloc[i]
    
    SI = row['Irradiance']
    shading_factor = row['SelfShading']
    
    # 한 래스터 파일 처리 및 통계 계산 (tif 파일은 별도 출력하지 않음)
    nonzero_count, total_value = process_raster_file(raster_path, SI, shading_factor)
    
    # 통계 정보 리스트에 추가: 파일명, 0이 아닌 셀 갯수, 전체 값 합계
    stats_list.append([raster_file, nonzero_count, total_value])

# ------------------------------------
# 5. 통계 결과를 새로운 CSV 파일로 저장
stats_df = pd.DataFrame(stats_list, columns=['RasterFile', 'NonZeroCount', 'TotalValue'])
stats_csv_path = os.path.join(output_folder, "raster_statistics.csv")
stats_df.to_csv(stats_csv_path, index=False, encoding='utf-8')

print(f"모든 래스터 파일의 발전량 통계 CSV 생성 및 {output_folder}에 저장 완료!")

래스터 처리중: 100%|██████████| 4400/4400 [01:59<00:00, 36.71it/s]


모든 래스터 파일의 발전량 통계 CSV 생성 및 C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Power_naju_Hori에 저장 완료!


In [17]:
import pandas as pd
import os


NH=4600 
NV=10490
GH=6300
GV=6700

# 1. CSV 입력 및 출력 경로 설정
csv_input = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\tmy\Study_tmy\Gangneung_samcsv.csv"
csv_output = r"C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Power_gangneung_Hori\PVgen_GHori.csv"

# 2. 헤더가 3번째 행에 있으므로 header=2로 읽기
df = pd.read_csv(csv_input, encoding='utf-8', header=2)

# 3. 필요한 열 추출: 'Irradiance', 'Hable', 'Month', 'Day', 'Hour'
required_cols = ['Irradiance', 'Hable', 'Month', 'Day', 'Hour']
if not all(col in df.columns for col in required_cols):
    raise ValueError("필요한 열이 누락되어 있습니다.")
df_filtered = df[required_cols].copy()

# 4. 발전량 계산
# PVgen = 0.22 * 0.9 * Irradiance * Hable * 4500 * 20
df_filtered['PVgen'] = 0.22 * 0.9 * df_filtered['Irradiance'] * df_filtered['Hable'] * GH * 20

# 5. Timestamp 문자열 생성 (year는 모두 2023, Month, Day, Hour는 두 자리 형식)
df_filtered['Timestamp'] = (
    '2023-' +
    df_filtered['Month'].astype(int).astype(str).str.zfill(2) + '-' +
    df_filtered['Day'].astype(int).astype(str).str.zfill(2) + '-' +
    df_filtered['Hour'].astype(int).astype(str).str.zfill(2)
)

# 6. 결과 DataFrame 구성
# A열: Time (Timestamp), B열: PVgen
result_df = df_filtered[['Timestamp', 'PVgen']].copy()
result_df.columns = ['Time', 'PVgen']

# 7. 추가 기능: 
# 7.1 C열 ("kWh"): PVgen의 0.001배 값
result_df['kWh'] = result_df['PVgen'] * 0.001

# 7.2 D열 ("kWh/m2"): kWh 값을 4500*20으로 나눈 값
result_df['kWh/m2'] = result_df['kWh'] /  (GH * 20)

# 8. CSV 파일로 출력
result_df.to_csv(csv_output, index=False, encoding='utf-8')

print(f"CSV 파일이 생성되었습니다: {csv_output}")

CSV 파일이 생성되었습니다: C:\Users\82105\Desktop\Junkyo\2025\PVSEC\GIS_preprocessing\Power_gangneung_Hori\PVgen_GHori.csv
